# Einführung in Python für Data Analytics
## Abschlussproket
### Dozent: Dr. Alessandro Bramucci
### Bearbeitung und Präsentation:
### Jonathan Willrich, Gregor Henning, Sebastian Knoche

## Importieren der Bibliotheken

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

## Importieren der Daten

```python
# speichere den relativen Pfad und den Dateinamen der Quelle 
file_besucher = 'data/besucher.csv'
file_geo = 'data/geo.txt'
file_kunden = 'data/kunden.csv'

# importiere Daten aus der Quelle in Dataframes
df_besucher = pd.read_csv(file_besucher, sep=";", decimal = ',')
df_geo = pd.read_table(file_geo, decimal = '.')
df_kunden = pd.read_csv(file_kunden, sep=";", decimal = '.')
```

In [2]:
# speichere den relativen Pfad und den Dateinamen der Quelle 
file_besucher = 'data/besucher.csv'
file_geo = 'data/geo.txt'
file_kunden = 'data/kunden.csv'

# importiere Daten aus der Quelle in Dataframes
df_besucher = pd.read_csv(file_besucher, sep=";", decimal = ',')
df_geo = pd.read_table(file_geo, decimal = '.')
df_kunden = pd.read_csv(file_kunden, sep=";", decimal = '.')

## Bereinigen der Daten

### Analyse von df_besucher

In [3]:
df_besucher.isnull().sum()

Alter         0
Einkommen     0
Geschlecht    0
Zeit          0
KundeNr       0
dtype: int64

In [4]:
df_besucher.dtypes

Alter           int64
Einkommen     float64
Geschlecht      int64
Zeit          float64
KundeNr        object
dtype: object

In [5]:
df_besucher.head(5)

,Alter,Einkommen,Geschlecht,Zeit,KundeNr
0,30,39919.095335,0,29.0,K1814
1,41,53430.673086,0,27.0,K1544
2,64,73163.216927,0,32.0,K1945
3,41,36761.482581,0,44.0,K1825
4,35,42684.331632,1,33.0,K1144


### Datenmodelierung für df_besucher

```python
# Um das spätere Zusammenführen zu erleichtern, sind folgende Anpassungen hilfreich:

# Füge eine Spalte "Kunde" hinzu, die für Besucher 0 ist
df_besucher['Kunde'] = 0

# Füge die Spalte "Preis" hinzu und setze die Werte auf 0
df_besucher.insert(2, 'Preis', 0)

# Füge die Spalte "Ausreißer" hinzu und setze die Werte auf False
df_besucher.insert(6, 'Ausreißer', False)
```

In [6]:
# Um das spätere Zusammenführen zu erleichtern, sind folgende Anpassungen hilfreich:

# Füge eine Spalte "Kunde" hinzu, die für Besucher 0 ist
df_besucher['Kunde'] = 0

# Füge die Spalte "Preis" hinzu und setze die Werte auf 0
df_besucher.insert(2, 'Preis', 0)

# Füge die Spalte "Ausreißer" hinzu und setze die Werte auf False
df_besucher.insert(6, 'Ausreißer', False)

### Analyse von df_geo

In [7]:
df_geo.isnull().sum()

KundeNr          0
Niederlassung    0
dtype: int64

In [8]:
df_geo.dtypes

KundeNr          object
Niederlassung    object
dtype: object

In [9]:
df_geo.head(5)

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,NRW
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern


### Datenmodelierung für df_geo

```python
# Ersetze alle Instanzen von "NRW" in der Spalte "Niederlassung" mit "Nordrhein-Westfalen"
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Nordrhein-Westfalen' if 'NRW' in x else x)

# Ersetze alle Einträge, die "Berlin" in der Spalte "Niederlassung" enthalten, mit "Berlin"
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Berlin' if 'berlin' in x.lower() else x)
```

In [10]:
# Ersetze alle Instanzen von "NRW" in der Spalte "Niederlassung" mit "Nordrhein-Westfalen"
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Nordrhein-Westfalen' if 'NRW' in x else x)

# Ersetze alle Einträge, die "Berlin" in der Spalte "Niederlassung" enthalten, mit "Berlin"
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Berlin' if 'berlin' in x.lower() else x)

### Analyse von df_kunden

In [11]:
df_kunden.isnull().sum()

Alter         0
Einkommen     0
Preis         0
Geschlecht    5
Zeit          0
KundeNr       0
dtype: int64

In [12]:
df_kunden.dtypes

Alter           int64
Einkommen     float64
Preis         float64
Geschlecht    float64
Zeit          float64
KundeNr        object
dtype: object

In [13]:
df_kunden.head(5)

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.0,88160.31,1.0,43.0,K0310
1,54,77644.0,103145.70,1.0,40.0,K1042
2,55,44341.0,80565.16,0.0,37.0,K0382
3,49,67271.0,83949.89,0.0,42.0,K0498
4,46,49832.0,93781.58,0.0,41.0,K0552


### Datenmodelierung für df_kunden

```python
# Berechne den Modus für die Werte in der Spalte "Geschlecht"
geschlecht_mode = df_kunden['Geschlecht'].mode()[0]
```

In [14]:
# Berechne den Modus für die Werte in der Spalte "Geschlecht"
geschlecht_mode = df_kunden['Geschlecht'].mode()[0]
print(f'Der Modus ist {geschlecht_mode}.')

Der Modus ist 1.0.


```python
# Ersetze alle leeren Einträge in der Spalte "Geschlecht" mit dem Modus der Werte in der Spalte "Geschlecht"
df_kunden['Geschlecht'].fillna(geschlecht_mode, inplace=True)

# Konvertiere die Spalte "Geschlecht" in den entsprechenden Integer-Datentyp
df_kunden['Geschlecht'] = df_kunden['Geschlecht'].astype(int)
```

In [15]:
# Replace all empty entries in the column "Geschlecht" with the mode of the values in the column "Geschlecht" 
df_kunden['Geschlecht'].fillna(geschlecht_mode, inplace = True)

# Convert column "Geschlecht" to the more propper integer date type
df_kunden['Geschlecht'] = df_kunden['Geschlecht'].astype(int)

```python
# Füge eine Spalte "Kunde" hinzu, die für Kunden 1 ist
df_kunden['Kunde'] = 1
```

In [16]:
# Füge eine Spalte "Kunde" hinzu, die für Kunden 1 ist
df_kunden['Kunde'] = 1

### Weitere Datenmodelierung für df_kunden, um Ausreißer zu entfernen

```python
# Einkommen:
# Berechnung von eink_Q1 (25. Perzentil), eink_Q3 (75. Perzentil) und eink_IQR
eink_Q1 = df_kunden['Einkommen'].quantile(0.25)
eink_Q3 = df_kunden['Einkommen'].quantile(0.75)
eink_IQR = eink_Q3 - eink_Q1

# Berechnung der Ausreißer-Grenzen
eink_untere_grenze = eink_Q1 - 1.5 * eink_IQR
eink_obere_grenze = eink_Q3 + 1.5 * eink_IQR

# Identifizierung von Ausreißern
df_kunden['Ausreißer'] = (df_kunden['Einkommen'] < eink_untere_grenze) | (df_kunden['Einkommen'] > eink_obere_grenze)
```

In [17]:
# Einkommen:
# Berechnung von eink_Q1 (25. Perzentil), eink_Q3 (75. Perzentil) und eink_IQR
eink_Q1 = df_kunden['Einkommen'].quantile(0.25)
eink_Q3 = df_kunden['Einkommen'].quantile(0.75)
eink_IQR = eink_Q3 - eink_Q1

# Berechnung der Ausreißer-Grenzen
eink_untere_grenze = eink_Q1 - 1.5 * eink_IQR
eink_obere_grenze = eink_Q3 + 1.5 * eink_IQR

# Identifizierung von Ausreißern
df_kunden['Ausreißer'] = (df_kunden['Einkommen'] < eink_untere_grenze) | (df_kunden['Einkommen'] > eink_obere_grenze)

```python
# Berechne den Median für die Werte in der Spalte "Einkommen"
eink_median = df_kunden['Einkommen'].median()
```

In [18]:
# Berechne den Median für die Werte in der Spalte "Einkommen"
eink_median = df_kunden['Einkommen'].median()

print(f'Der Median ist {eink_median:.2f} EUR.')

Der Median ist 59013.00 EUR.


```python
# Ersetze extreme Werte mit dem Median der Spalte "Einkommen"
df_kunden.loc[df_kunden['Ausreißer'] == True, 'Einkommen'] = eink_median
```

In [19]:
# Ersetze extreme Werte mit dem Median der Spalte "Einkommen"
df_kunden.loc[df_kunden['Ausreißer'] == True, 'Einkommen'] = eink_median

```python
# Alter:
# Berechnung von alt_Q1 (25. Perzentil), alt_Q3 (75. Perzentil) und alt_IQR
alt_Q1 = df_kunden['Alter'].quantile(0.25)
alt_Q3 = df_kunden['Alter'].quantile(0.75)
alt_IQR = alt_Q3 - alt_Q1

# Berechnung der Ausreißer-Grenzen
alt_untere_grenze = alt_Q1 - 1.5 * alt_IQR
alt_obere_grenze = alt_Q3 + 1.5 * alt_IQR

# Identifizierung von Ausreißern
df_kunden['Ausreißer'] = (df_kunden['Alter'] < alt_untere_grenze) | (df_kunden['Alter'] > alt_obere_grenze)
```

In [20]:
# Alter:
# Berechnung von alt_Q1 (25. Perzentil), alt_Q3 (75. Perzentil) und alt_IQR
alt_Q1 = df_kunden['Alter'].quantile(0.25)
alt_Q3 = df_kunden['Alter'].quantile(0.75)
alt_IQR = alt_Q3 - alt_Q1

# Berechnung der Ausreißer-Grenzen
alt_untere_grenze = alt_Q1 - 1.5 * alt_IQR
alt_obere_grenze = alt_Q3 + 1.5 * alt_IQR

# Identifizierung von Ausreißern
df_kunden['Ausreißer'] = (df_kunden['Alter'] < alt_untere_grenze) | (df_kunden['Alter'] > alt_obere_grenze)

```python
# Berechne den Median für die Werte in der Spalte "Alter"
alt_median = df_kunden['Alter'].median()

print(f'Der Median ist {alt_median}.')
```

In [21]:
# Berechne den Median für die Werte in der Spalte "Alter"
alt_median = df_kunden['Alter'].median()

print(f'Der Median ist {alt_median:.2f} Jahre.')

Der Median ist 51.00 Jahre.


```python
# Ersetze extreme Werte mit dem Median der Spalte "Alter"
df_kunden.loc[df_kunden['Ausreißer'] == True, 'Alter'] = alt_median
```

In [22]:
# Ersetze extreme Werte mit dem Median der Spalte "Alter"
df_kunden.loc[df_kunden['Ausreißer'] == True, 'Alter'] = alt_median

### Zusammenführen von df_besucher und df_geo

```python
# Zusammenführen von df_besucher und df_geo basierend auf KundeNr
df_besucher_geo = pd.merge(df_besucher, df_geo, on='KundeNr', how='left')

# Zusammenführen von df_kunden und df_geo basierend auf KundeNr
df_kunden_geo = pd.merge(df_kunden, df_geo, on='KundeNr', how='left')
```

In [23]:
# Zusammenführen von df_besucher und df_geo basierend auf KundeNr
df_besucher_geo = pd.merge(df_besucher, df_geo, on='KundeNr', how='left')

# Zusammenführen von df_kunden und df_geo basierend auf KundeNr
df_kunden_geo = pd.merge(df_kunden, df_geo, on='KundeNr', how='left')

```python
# Zusammenführen von df_besucher_geo und df_kunden_geo
df_gesamt = pd.concat([df_besucher_geo, df_kunden_geo], ignore_index=True)
```

In [24]:
# Zusammenführen von df_besucher_geo und df_kunden_geo
df_gesamt = pd.concat([df_besucher_geo, df_kunden_geo], ignore_index=True)

## Fragen

### 1. Wie viele Autos wurden verkauft?

```python
# Filtern der Verkäufe
df_verk = df_gesamt[df_gesamt['Kunde'] == True]

# Anzahl der Verkäufe ermitteln
anzahl_verk = len(df_verk)
```

In [25]:
# Filtern der Verkäufe
df_verk = df_gesamt[df_gesamt['Kunde'] == True]

# Anzahl der Verkäufe ermitteln
anzahl_verk = len(df_verk)
print(f'Anzahl der verkauften Autos: {anzahl_verk} Stk.')

Anzahl der verkauften Autos: 1104 Stk.


### 2. Was ist der Höchst-, Mindest- und Durchschnittspreis der verkauften Autos?

```python
# Höchster Verkaufspreis
max_preis = df_verk['Preis'].max()

# Niedrigster Verkaufspreis
min_preis = df_verk['Preis'].min()

# Durchschnittspreis
mittel_preis = df_verk['Preis'].mean()
```

In [26]:
# Höchster Verkaufspreis
max_preis = df_verk['Preis'].max()

# Niedrigster Verkaufspreis
min_preis = df_verk['Preis'].min()

# Durchschnittspreis
mittel_preis = df_verk['Preis'].mean()

# Ausgabe der Ergebnisse
print(f'''Höchster Verkaufspreis:   {max_preis:.2f} EUR
Niedrigster Verkaufspreis: {min_preis:.2f} EUR
Durchschnittspreis:        {mittel_preis:.2f} EUR''')

Höchster Verkaufspreis:   165482.31 EUR
Niedrigster Verkaufspreis: 21471.65 EUR
Durchschnittspreis:        88975.20 EUR


### 3. Wie hoch war der Gesamtumsatz?

```python
# Gesamtumsatz
gesamtumsatz = df_verk['Preis'].sum()
```

In [27]:
# Gesamtumsatz
gesamtumsatz = df_verk['Preis'].sum()
print(f'Gesamtumsatz: {gesamtumsatz:.2f} EUR')

Gesamtumsatz: 98228619.38 EUR


### 4. Wie viele Autos wurden pro Bundesland verkauft?

```python
# Liste der Bundesländer mit der Anzahl der Verkäufe
anzahl_verk_niederl = df_verk['Niederlassung'].value_counts()

# Alphabetisch sortieren
anzahl_verk_niederl = anzahl_verk_niederl.sort_index()
```

In [28]:
# Liste der Bundesländer mit der Anzahl der Verkäufe
anzahl_verk_niederl = df_verk['Niederlassung'].value_counts()

# Alphabetisch sortieren
anzahl_verk_niederl = anzahl_verk_niederl.sort_index()

# Ausgabe
print(f'Anzahl der Verkäufe pro Bundesland in EUR:\n{anzahl_verk_niederl}')

Anzahl der Verkäufe pro Bundesland in EUR:
Niederlassung
Baden-Württemberg      205
Bayern                 188
Berlin                 126
Brandenburg             37
Düsseldorf               1
Hamburg                 43
Hessen                  96
Niedersachsen           96
Nordrhein-Westfalen    174
Sachsen                 89
Thüringen               49
Name: count, dtype: int64


### 5. Wie hoch war der durchschnittliche Umsatz pro Bundesland?

```python
# Durchschnittlicher Umsatz pro Bundesland
umsatz_pro_niederl = df_verk.groupby('Niederlassung')['Preis'].mean()

# Runden auf zwei Nachkommastellen
umsatz_pro_niederl = umsatz_pro_niederl.round(2)
```

In [29]:
# Durchschnittlicher Umsatz pro Bundesland
umsatz_pro_niederl = df_verk.groupby('Niederlassung')['Preis'].mean()

# Runden auf zwei Nachkommastellen
umsatz_pro_niederl = umsatz_pro_niederl.round(2)

# Sortieren nach dem durchschnittlichem Umsatz (absteigend)
#umsatz_pro_niederl = umsatz_pro_niederl.sort_values(ascending=False)

# Ausgabe
print(f'Durchschnittlicher Umsatz pro Bundesland in EUR:\n{umsatz_pro_niederl}')

Durchschnittlicher Umsatz pro Bundesland in EUR:
Niederlassung
Baden-Württemberg       90969.56
Bayern                  89344.31
Berlin                  89584.98
Brandenburg             86673.64
Düsseldorf             120215.50
Hamburg                 85549.50
Hessen                  85864.93
Niedersachsen           87963.31
Nordrhein-Westfalen     88451.76
Sachsen                 89455.08
Thüringen               90816.99
Name: Preis, dtype: float64


### 6. In welchem Bundesland wurde das teuerste Auto verkauft?

```python
# Ermittlung der höchsten Verkaufspreise für jedes Bundesland
max_pro_niederl = df_verk.groupby('Niederlassung')['Preis'].max()

# Finden des Bundeslandes mit dem höchsten Verkaufspreis
max_niederl = max_pro_niederl.idxmax()
max_preis = max_pro_niederl.max()
```

In [30]:
# Ermittlung der höchsten Verkaufspreise für jedes Bundesland
max_pro_niederl = df_verk.groupby('Niederlassung')['Preis'].max()

# Finden des Bundeslandes mit dem höchsten Verkaufspreis
max_niederl = max_pro_niederl.idxmax()
max_preis = max_pro_niederl.max()

print(f'Das Bundesland mit dem höchsten Verkaufspreis ist {max_niederl} mit einem Preis von {max_preis} EUR.')

Das Bundesland mit dem höchsten Verkaufspreis ist Niedersachsen mit einem Preis von 165482.31 EUR.


### 7. Haben mehr Frauen oder mehr Männer unsere Autos gekauft?

```python
# Ermittlung der Anzahl der Verkäufe pro Geschlecht
verk_pro_geschl = df_verk.groupby('Geschlecht')['Geschlecht'].count()

# Finden des Geschlechts mit den meisten Käufen
max_geschl = 'Männer' if verk_pro_geschl.idxmax() else 'Frauen'
max_verk = verk_pro_geschl.max()
```

In [31]:
# Ermittlung der Anzahl der Verkäufe pro Geschlecht
verk_pro_geschl = df_verk.groupby('Geschlecht')['Geschlecht'].count()

# Finden des Geschlechts mit den meisten Käufen
max_geschl = 'Männern' if verk_pro_geschl.idxmax() else 'Frauen'
max_verk = verk_pro_geschl.max()

print(f'Die meisten Autos wurden von {max_geschl} gekauft, sie haben {max_verk} Autos gekauft.')

Die meisten Autos wurden von Männern gekauft, sie haben 706 Autos gekauft.


### 8. Wie hoch ist das Durchschnittsalter unserer Kunden? Wie hoch ist das Durchschnittsalter unserer Besucher?

```python
# Berechnung des Durchschnittsalters für Kunden und Besucher
mittel_alt_kunden = df_gesamt[df_gesamt['Kunde'] == 1]['Alter'].mean()
mittel_alt_besucher = df_gesamt[df_gesamt['Kunde'] == 0]['Alter'].mean()
```

In [32]:
# Berechnung des Durchschnittsalters für Kunden und Besucher
mittel_alt_kunden = df_gesamt[df_gesamt['Kunde'] == 1]['Alter'].mean()
mittel_alt_besucher = df_gesamt[df_gesamt['Kunde'] == 0]['Alter'].mean()

print(f'''Das Durchschnittsalter der Kunden ist {mittel_alt_kunden:.2f} Jahre.
Das Durchschnittsalter der Besucher ist {mittel_alt_besucher:.2f} Jahre.''')

Das Durchschnittsalter der Kunden ist 50.38 Jahre.
Das Durchschnittsalter der Besucher ist 49.88 Jahre.


### 9. Wie hoch ist das Durchschnittseinkommen unserer Kunden? Wie hoch ist das Durchschnittseinkommen unserer Besucher?

```python
# Berechnung der Durchschnittseinkommen für Kunden und Besucher
mittel_eink_kunden = df_gesamt[df_gesamt['Kunde'] == 1]['Einkommen'].mean()
mittel_eink_besucher = df_gesamt[df_gesamt['Kunde'] == 0]['Einkommen'].mean()

print(f'''Das Durchschnittseinkommen der Kunden ist {mittel_eink_kunden:.2f} EUR.
Das Durchschnittseinkommen der Besucher ist {mittel_eink_besucher:.2f} EUR.''')
```

In [33]:
# Berechnung der Durchschnittseinkommen für Kunden und Besucher
#mittel_eink_kunden = df_gesamt[df_gesamt['Kunde'] == 1]['Einkommen'].mean()
#mittel_eink_besucher = df_gesamt[df_gesamt['Kunde'] == 0]['Einkommen'].mean()

#print(f'''Das Durchschnittseinkommen der Kunden ist {mittel_eink_kunden:.2f} EUR.
#Das Durchschnittseinkommen der Besucher ist {mittel_eink_besucher:.2f} EUR.''')

In [34]:
eink_kunden = df_gesamt[df_gesamt['Kunde'] == 1]['Einkommen']
eink_besucher = df_gesamt[df_gesamt['Kunde'] == 0]['Einkommen']

# Berechnung der Durchschnittseinkommen für Kunden und Besucher
mittel_eink_kunden = eink_kunden.mean()
mittel_eink_besucher = eink_besucher.mean()

print(f'''Das Durchschnittseinkommen der Kunden ist {mittel_eink_kunden:.2f} EUR.
Das Durchschnittseinkommen der Besucher ist {mittel_eink_besucher:.2f} EUR.''')

Das Durchschnittseinkommen der Kunden ist 58850.91 EUR.
Das Durchschnittseinkommen der Besucher ist 59367.94 EUR.


### 10. Bestimmen Sie, ob es einen statistisch signifikanten Unterschied zwischen dem Durchschnittseinkommen der Kunden und dem Durchschnittseinkommen der Besucher gibt.
#### Sie müssen einen Zweistichproben-t-Test durchführen. Beginnen Sie mit der Aufstellung der Null- und Alternativhypothese. Verwenden Sie ein Signifikanzniveau (α) von 5%. Nehmen Sie an, dass die Varianzen der beiden Gruppen gleich sind.

Nullhypothese (H0​): Es gibt keinen Unterschied zwischen dem Durchschnittseinkommen der Kunden und der Besucher. (μKunden=μBesucher)  
Alternativhypothese (H1​): Es gibt einen Unterschied zwischen dem Durchschnittseinkommen der Kunden und der Besucher. (μKunden≠μBesucher​)  
Wir verwenden ein Signifikanzniveau (α) von 5%.

```python
# Zweistichproben-t-Test durchführen, unter der Annahme, dass die Varianzen gleich sind
t_stat, p_value = stats.ttest_ind(eink_kunden, eink_besucher, equal_var=True)

print(f'''Ergebnisse des Zweistichproben-t-Tests:
t-Statistik: {t_stat:.2f}
p-Wert: {p_value:.5f}''')

# Entscheidung basierend auf dem p-Wert
alpha = 0.05
if p_value < alpha:
    print('Es gibt einen statistisch signifikanten Unterschied zwischen dem Durchschnittseinkommen der Kunden und dem der Besucher.')
else:
    print('Es gibt keinen statistisch signifikanten Unterschied zwischen dem Durchschnittseinkommen der Kunden und dem der Besucher.')
```

In [36]:
# Zweistichproben-t-Test durchführen, unter der Annahme, dass die Varianzen gleich sind
t_stat, p_value = stats.ttest_ind(eink_kunden, eink_besucher, equal_var=True)

print(f'''Ergebnisse des Zweistichproben-t-Tests:
t-Statistik: {t_stat:.2f}
p-Wert: {p_value:.5f}''')

# Entscheidung basierend auf dem p-Wert
alpha = 0.05
if p_value < alpha:
    print('Es gibt einen statistisch signifikanten Unterschied zwischen dem Durchschnittseinkommen der Kunden und dem der Besucher.')
else:
    print('Es gibt keinen statistisch signifikanten Unterschied zwischen dem Durchschnittseinkommen der Kunden und dem der Besucher.')

Ergebnisse des Zweistichproben-t-Tests:
t-Statistik: -0.70
p-Wert: 0.48183
Es gibt keinen statistisch signifikanten Unterschied zwischen dem Durchschnittseinkommen der Kunden und dem der Besucher.


# Vielen Dank für Ihre Aufmerksamkeit 🐍 